<a href="https://colab.research.google.com/github/rdale02/Back-From-The-Brink/blob/master/SmallWorldChallenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

SMALL WORLD:
REVEAL 1 MONSTER IN YOUR HAND
REVEAL 1 MONSTER IN YOUR DECK WITH THE SAME TYPE, ATTRIBUTE, ATK OR DEF (EXACTLY ONE) AS THE FIRST MONSTER
ADD 1 MONSTER FROM YOUR DECK TO YOUR HAND WITH THE SAME TYPE, ATTRIBUTE, ATK OR DEF (EXACTLY ONE) AS THE SECOND MONSTER

In [ ]:
import pandas as pd
deck = getDeck("/content/SAMPLE.csv")
card_set = deck.drop_duplicates() # ONE OF EVERY CARD IN THE DECK -ALSO USED FOR REFERENCE IN THE SECOND PASS
df = secondPass(firstPass()).drop_duplicates()
df.reset_index(drop=True)
df.to_csv("output.csv")


FIRST CHECK:
FOR EACH RECORD IN THE DECK, THE CARD IN YOUR HAND GETS COMPARED AGAINST IT FOR MATCHING FEATURES.

REMOVE ALL INSTANCES OF THE CARD YOU HAVE IN HAND FROM THE POOL OF SEARCH CARDS

In [181]:
def filter_deck(name):
  return deck.drop(deck[deck['NAME'] == name].index)

COMPARE THE TWO CARDS TO SEE IF THERE EXISTS EXACTLY ONE COMMON PROPERTY BETWEEN THEM

In [200]:
def compare(name1, name2):
  first = card_set[card_set['NAME']==name1].reset_index(drop=True)
  second = card_set[card_set['NAME']==name2].reset_index(drop=True)
  common_properties = 0
  if(first['ATK'][0] == second['ATK'][0]):
    common_properties += 1
  if(first['DEF'][0] == second['DEF'][0]):
    common_properties += 1
  if(first['TYPE'][0] == second['TYPE'][0]):
    common_properties += 1 
  if(first['ATTRIBUTE'][0] == second['ATTRIBUTE'][0]):
    common_properties += 1 
  if(first['LEVEL'][0] == second['LEVEL'][0]):
    common_properties += 1
  return (common_properties == 1)

In [198]:
def firstPass():
  results = pd.DataFrame(columns=['HAND','DECK','SEARCH'])
  end_results = pd.DataFrame(columns=['HAND','DECK','SEARCH'])
  for i, row in card_set.iterrows(): # for each available card
    filtered_deck = filter_deck(card_set['NAME'][i])
    for j, row in filtered_deck.iterrows():
      if(compare(card_set['NAME'][i], filtered_deck['NAME'][j])):
        results = results.append(pd.DataFrame([[card_set['NAME'][i],filtered_deck['NAME'][j]]],columns=['HAND','DECK'])).reset_index(drop=True)
  return results

TAKE THE RESULTS OF THE FIRST PASS AS CHAIN LINKS. YOU HAVE THE FIRST AND SECOND LINKS IN THE CHAIN; YOU JUST NEED THE THIRD.
FOR EACH COMBINATION OF CARDS RETURNED IN THE FIRST PASS, YOU NOW NEED TO SHOW ALL THE CARDS THAT THEY CAN PULL OUT OF THE DECK TOGETHER.
FOR EXAMPLE, 'GHOST BELLE & HAUNTED MANSION' CAN BE USED TO BANISH 'EFFECT VEILER' FROM THE DECK SINCE BOTH HAVE 0 ATK. NOW THAT 'EFFECT VEILER' IS BANISHED, YOU CAN SEARCH FOR 'BACKUP SECRETARY' BECAUSE BOTH OF THESE CARDS ARE 'LIGHT' ATTRIBUTE. 

In [201]:
def secondPass(results):
  end_results = pd.DataFrame(columns=['HAND','DECK','SEARCH'])
  for i, row in results.iterrows():
    print(row)
    # check the card in Results against the cards in the deck
    filtered_deck = filter_deck(results['DECK'][i])
    for j, row in filtered_deck.iterrows():
      print(results['DECK'][i],filtered_deck['NAME'][j],compare(results['DECK'][i],filtered_deck['NAME'][j]))
      if (compare(results['DECK'][i],filtered_deck['NAME'][j])):
        end_results = end_results.append(pd.DataFrame([[results['HAND'][i], results['DECK'][i],filtered_deck['NAME'][j]]],columns=['HAND','DECK','SEARCH'])).reset_index(drop=True)
  return end_results
 

In [41]:
def getDeck(path):
  return pd.read_csv(path)